# 6. Results

This notebook visualizes some results collected over the course of this work. For this, I created
reports in my `Weights and Biases` instance, but I couldn't find a way to embed them here except
with images. I would've made them public, but I switched servers and platforms after this work, so I
hope this is fine for now.

In [ ]:
from IPython.display import Image
Image(filename='../img/6-1-dataset-impacts.png')

This image shows the impact of position embeddings. Without using them with the 5s snippets, the
model converges faster in the beginning. Furthermore, the model has a better performance from the
very beginning. Also, it is not able to reach a performance close to the model using the pre-trained
position embeddings, regardless of the training hyperparameters.

From now on, only results from the dataset with a temporal dimension of 10.24 seconds are shown. 

In [ ]:
Image(filename='../img/6-2-using-dropout.png')

As you can see, applying dropout does not help the model to get a better performance or generalize
better. Probably because the default values for AST are 0 and modifying it leads to unknown 
internal behavior.

In [ ]:
Image(filename='../img/6-3-aggregation.png')

These charts clearly show that mean pooling and weighting of the snippet classification scores work
best for the final genre prediction of a song. Weighting gains more performance during the runs
while mean pooling has higher performance average at the end. That's why it is used as the primary
performance metric.

In [ ]:
Image(filename='../img/6-4-time-stretching.png')

While experimenting with augmentations, I realized, that there was a strange behavior in the
training loss. It originated from the time stretch augmentation, which led to unlearnable samples.
This is probably the case because many of my electronic genres also have a strong coupling with the
speed of the music and thus a difference leads to false predictions. Strangely, the loss didn't even
improve over the whole training process as the "all 2" run graph shows. This run used all kinds of
techniques such as weight decay, weighted loss, augmentation, and dropout. The green graph shows the
same setup but with some disabled augmentations as I gradually disabled them to find the cause of
the training loss problem. The third run "without stretch" shows the behavior as it should be, this
time only with the fixed augmentation.

In [ ]:
Image(filename='../img/6-5-augmentation-p.png')

Applying augmentations didn't made too much difference to the performance. Most notable was that the
run with 100 % augmentation was the run with the least loss on both, the validation and the test
set. But it was also the run with the least score. Clearly, the training loss is higher, the more
augmentation is used. This makes totally sense, since augmentation removes features or distorts them
on the training data.

Due to the significant decrease in loss and not so great difference in the final score, 100 %
augmentation is chosen for further runs.

In [ ]:
Image(filename='../img/6-6-weighted-loss.png')

The weighted loss should help to add more focus to the loss of under-represented classes and remove
it from over-represented. The experiments show that it does not help in this case. Neither the
under-represented classes get a better performance nor the over-represented classes.

In [ ]:
Image(filename='../img/6-7-learning-rates.png')

Different learning rates influence the performance and the training time of the model. Three
learning rates were tested: 1e-5, 5e-5, and 1e-4. The graphs show that 1e-5 takes way longer to
train but also has the lowest validation loss. 1e-4 and 5e-5 both only need 5 epochs to train. 1e-4
has comprehensible the lowest training loss, but therefore the highest losses on the validation and
test sets. Interestingly, its test loss is higher than with a learning rate of 1e-5, but the score
is nevertheless better. The best results came from the default and middle learning rate of 5e-5.
Is shows the best overall performance while also having the lowest loss on the test set and only
needs five epochs to train. This one is used for further training runs.

In [ ]:
Image(filename='../img/6-8-weight-decay.png')

The weight decay can help the model to generalize better because it reduces overfitting. The weights
are decayed each step, which means that the model has to always move all weights in correct
directions. On this dataset, weight decay seemed to work better on the validation set but was quite
worse on the test split. But this time, we can clearly see its impact on the under- and
over-represented genres. The first 25th percentile of the genres got an improvement of around 5.1 %
while the last 25th percentile only got a decrease of around 2 %. Thus, weight decay is proven
helpful for under-represented classes while (of course) hurt the overall test loss and score a
little but not too much.

In [ ]:
Image(filename='../img/6-9-balanced-dataset.png')

A balanced dataset clearly improves the performance on the less represented genres while also
loosing some accuracy on the higher represented ones, which is a reasonable result. Thus, the
overall loss on the test set is higher and the score is lower.

In [ ]:
Image(filename='../img/6-10-song-models.png')

This report shows the different approaches to classify whole songs by using the CLS-tokens of their
six snippets. As you can see, the best model just uses the mean of them, which is straightforward.
When applying more complex approaches, the loss is higher and the score is lower. Probably due to
too less training data. But overall, they all perform quite well, always with an accuracy greater
than 84 %.

## Final Report and Conclusion

In [ ]:
Image(filename='../img/6-11-best-model.png')

This report summarizes the results of this work. It shows the best models and different techniques.

The balanced dataset performs bad on all metrics except for the under-represented genres.

Interestingly, the more complex a model, the nearer it gets to the run which used the balanced
dataset. For example, you can see that the model using self attention on the CLS tokens and
introduced a song-wide CLS token performs better on the first 25th percentile than other runs, but
also worse on the last 25th percentile. But not as extreme as the balanced dataset run. Furthermore,
while it shows the worst overall accuracy, it still has a good accuracy since all runs in this
report worked quite good (84 % - 88 %). Note that, although it has the worst performance, the loss
on the test set is remarkably low, which means that this model is able to generalize better as it
also shows in the percentile graphs. It also gains better scores at the precision, recall, and f1
metrics than all the other runs except the one using the means of the CLS tokens.

You can see that all the song models, which are more complex than the others, have a lower accuracy
than the both best models, which use aggregation of the predictions. But they also have a lower loss
and higher scores on the precision, recall, and f1 metrics. They are generally more able to
generalize. While the model using self attention moves more into the direction of the balanced
dataset run, the run using the mean of the CLS tokens seems to work best for this setup, providing
the best trade-offs.

The models using aggregation of the predictions are good learners. They provide the highest accuracy
on the test set but fall behind on important metrics as precision and recall. Also, their loss is
higher. In the percentile analysis, you can see that they learn the highly-represented classes
better as thus reach their accuracy. But they have a far lower performance on the first 25th
percentile accuracy (around 48 % instead of 60% with self-attention). At the last 25th percentile,
they only outperform by 6 % (88 % instead of 82 % with self-attention).

To conclude, no model will be used in production as the results didn't reach a stable accuracy on
all genres above 90%. But it was exciting to experiment with this task and a joy to improve and
analyze the model step by step, while making a bunch of mistakes and correcting them. Which model
is the best depends on the use case and for me, I would choose the one using the mean of the CLS
tokens as it seems to provide the best trade-offs at both ends as well as accuracy.

I hope you enjoyed this little journey into AST for (highly opinionated) genre classification!